In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
model = "1호모델의모델"

# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Y"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명
model_name = "{}_{}_k{}".format(model, freq, num_k)
model_name

'1호모델의모델_Y_k3'

In [3]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트.csv", index_col=0)

In [4]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [5]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [6]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [7]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [8]:
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

In [9]:
drop_togehter = tools.dict_data_drop(list_data.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)
scaled_together["2012-01"].columns

Index(['BW금액', 'CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '유상증자금액', '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액',
       '최대주주변경카운트', '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [10]:
pd.set_option('display.max_columns', 130)

In [11]:
# 최초 모델을 돌리고
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=100, raw_data=scaled_together.copy())

100%|██████████| 100/100 [00:07<00:00, 13.23it/s]


In [12]:
# 최초모델의 결과
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict,
                                                            raw_data = scaled_together, 
                                                            label_data=label_data_pp_man, freq=freq)

# 모델의 모델(나쁜놈들도 다시한번 굴림)
model2_save_dict, filtered_data = evaluate.cluster_of_cluster_model(model_save=model_save_dict,
                                  raw_data=scaled_together,
                                  eval_label=eval_one, num_k=3, max_sample=100)

100%|██████████| 100/100 [00:01<00:00, 57.13it/s]


### 모델의 모델 평가

In [13]:
size =evaluate.eval_cluster_size(model2_save_dict)
size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

340.0
103.0


,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,169.0,191.0,226.0,125.0,194.0,239.0,158.0,198.0,194.0,218.0
1,187.0,203.0,165.0,182.0,196.0,204.0,229.0,197.0,253.0,340.0
2,148.0,103.0,155.0,208.0,174.0,152.0,208.0,186.0,266.0,212.0
기업수,504.0,497.0,546.0,515.0,564.0,595.0,595.0,581.0,713.0,770.0


In [14]:
eval, eval_original = evaluate.eval_cluster_result(model2_save_dict, raw_data = filtered_data,
                                                    label_data=label_data_pp_man, freq=freq)

eval.to_csv("{}_관리종목.csv".format(model_name))
eval_original.to_csv("{}_관리종목전체대비.csv".format(model_name))

In [15]:
eval_t,eval_t_ori = evaluate.eval_cluster_result(model2_save_dict, raw_data = filtered_data, 
                                                 label_data=label_data_pp_war, freq=freq)

eval_t.to_csv("{}_투자환기종목.csv".format(model_name))
eval_t_ori.to_csv("{}_투자환기종목전체대비.csv".format(model_name))

In [16]:
print(evaluate.calculate_catch_probability(eval, eval_t, None))
print(evaluate.calculate_match_probability(eval, eval_t,None))
evaluate.calculate_catch_probability(eval_original, eval_t_ori, None)

0.6680415086079547
1.0


0.07013785828604684

In [17]:
def pct(df):
    return df / np.sum(df)

dist = evaluate.eval_cluster_distance(eval, filtered_data, model2_save_dict)
dist = dist.apply(pct, axis=1)
dist2= dist.mean(axis=0).sort_values(ascending=False)

dist.to_csv("{}_거리분해.csv".format(model_name))
dist2.to_csv("{}_거리분해평균.csv".format(model_name))

dist2

최대주주비율_변동성             0.204750
CB금액                   0.157729
차입금의존도                 0.144549
결산기간_거래량증가             0.108569
최대주주비율                 0.093785
최대주주변경카운트              0.041291
당기순이익                  0.036002
타법인주식및출자증권취득결정금액       0.033810
판매 및 관리비               0.032501
매출채권                   0.022618
조회공시요구카운트              0.022585
결산기간_주가변동률             0.018397
영업이익                   0.016892
불성실공시법인지정카운트           0.015686
최대주주변경을수반하는주식양수도금액     0.009521
영업현금흐름                 0.008707
유상증자금액                 0.007500
최대주주변경을수반하는주식담보제공금액    0.005824
총자본                    0.005186
상호변경카운트                0.004759
BW금액                   0.003764
매출총이익                  0.003339
경영권분쟁소송                0.001752
업종변경카운트                0.000465
특수관계인에대한자금대여금액         0.000017
dtype: float64

고위험 클러스터의 평균 재무, 공시 상황 출력

* eval_cluster Result의 결과를 받는다 / model_save_dict / raw_data -> drop_together (스케일 전 데이터를 받아야함)

In [27]:
# scaled data를 토대로 원본 데이터를 추출한다
key = filtered_data.keys()
original_data_dict = {}
for t in key:
    idx = filtered_data[t].index
    origianl_data = drop_togehter[t]
    original_data_dict[t] = origianl_data.loc[idx]

In [28]:
dict = evaluate.see_cluster_info(model2_save_dict, eval, original_data_dict)
data = pd.DataFrame()
col = dict.keys()

for t, df in dict.items():
    data = pd.concat([data, df], axis=1)

data.columns = col
data.T.to_csv("{}_평균정보.csv".format(model_name))
data.T

,BW금액,CB금액,경영권분쟁소송,불성실공시법인지정카운트,상호변경카운트,업종변경카운트,유상증자금액,조회공시요구카운트,최대주주변경을수반하는주식담보제공금액,최대주주변경을수반하는주식양수도금액,최대주주변경카운트,타법인주식및출자증권취득결정금액,특수관계인에대한자금대여금액,결산기간_거래량증가,결산기간_주가변동률,당기순이익,매출채권,매출총이익,영업이익,영업현금흐름,차입금의존도,총자본,최대주주비율,판매 및 관리비,최대주주비율_변동성
2012-01,2.964470e+09,5.254492e+08,0.000000,0.176471,0.069519,0.112299,5.242449e+09,0.871658,0.000000e+00,1.229126e+09,0.213904,3.049526e+09,149.732620,1.256659,1.576349,-3.100144e+06,2.363218e+07,1.296395e+06,-2.052738e+06,-5.499510e+05,30.104265,4.544850e+07,23.980396,3.349133e+06,-0.024039
2013-01,2.954361e+09,8.178217e+08,0.009852,0.103448,0.083744,0.029557,4.512339e+08,0.512315,0.000000e+00,1.170318e+09,0.167488,1.187566e+09,0.000000,inf,0.207505,-1.351623e+06,8.610562e+06,1.157579e+06,-4.956164e+05,-1.600387e+05,28.750382,2.954113e+07,32.530276,1.648039e+06,-0.009624
2014-01,2.687168e+08,2.947005e+09,0.022124,0.119469,0.066372,0.026549,9.997554e+09,0.464602,0.000000e+00,5.220796e+08,0.199115,2.149033e+09,306.946903,inf,2.145196,-1.318873e+07,3.751865e+07,-2.753013e+06,-6.360202e+06,-2.875537e+06,43.942301,3.381694e+07,24.684776,3.607189e+06,0.052031
2015-01,3.440000e+08,7.374460e+09,0.024000,0.216000,0.144000,0.088000,6.027502e+08,0.840000,6.098400e+08,4.848336e+09,0.440000,3.608146e+09,0.000000,2.010792,0.611897,-2.867222e+06,1.587111e+07,1.455653e+06,-1.401051e+06,-9.569295e+05,35.598160,3.610141e+07,23.826489,2.856704e+06,-0.031366
2016-01,1.116722e+09,1.549805e+10,0.464286,0.301020,0.198980,0.020408,9.855903e+09,0.877551,2.513090e+09,4.123536e+09,0.724490,5.340898e+09,737.489796,inf,1.063127,-8.097463e+06,1.950812e+07,1.412585e+05,-3.725497e+06,-1.097169e+06,34.629630,3.782605e+07,20.344377,3.866756e+06,inf
2017-01,6.163158e+08,1.844370e+10,0.361842,0.309211,0.157895,0.111842,7.191503e+09,0.644737,5.060245e+09,3.725409e+09,0.697368,5.002238e+09,0.000000,inf,0.894546,-6.283512e+06,1.714990e+07,1.522721e+06,-2.018482e+06,-1.460464e+06,35.032599,4.539889e+07,17.368013,3.541203e+06,inf
2018-01,5.807753e+09,2.707270e+10,0.531646,0.367089,0.183544,0.012658,2.571701e+09,0.481013,3.586237e+09,5.719362e+09,0.689873,9.546306e+09,0.000000,inf,1.641687,-4.551887e+06,1.979322e+07,1.734287e+06,-2.447031e+06,-1.958796e+06,33.784241,6.146192e+07,19.878133,4.181318e+06,0.005586
2019-01,2.354839e+09,2.069428e+10,0.198925,0.419355,0.193548,0.053763,1.448133e+09,0.387097,6.716283e+09,4.735273e+09,0.618280,9.062531e+09,0.000000,inf,0.492395,-4.100085e+06,1.187717e+07,1.909192e+06,-1.462864e+06,-1.093343e+06,36.207796,4.702668e+07,19.328319,3.372049e+06,inf
2020-01,1.952569e+09,1.565858e+10,0.407115,0.276680,0.162055,0.051383,5.655157e+09,0.300395,1.636247e+09,1.948938e+09,0.320158,7.976413e+09,3438.735178,inf,0.501169,-8.782394e+06,1.632716e+07,5.876500e+05,-4.757963e+06,-2.439076e+06,38.629486,7.992139e+07,20.830694,5.341926e+06,inf
2021-01,2.077982e+09,2.208451e+10,0.298165,0.243119,0.114679,0.013761,9.932791e+09,0.233945,3.062421e+09,1.248581e+09,0.339450,6.058384e+09,91.743119,inf,1.279406,-6.921556e+06,1.720878e+07,1.541121e+06,-3.940854e+06,-2.649400e+06,37.300975,7.964483e+07,21.744968,5.478895e+06,-0.004123


In [29]:
data.T.mean().to_csv("{}_평균정보의평균.csv".format(model_name))

In [30]:
# 그림뽑기

test = MyKmeans.visualize_3d_with_answer(filtered_data["2020-01"],
                                        model2_save_dict["2020-01"]["model"],
                                         answer_data=label_data_pp_man["2020-01"],
                                         fig_title="{}_관리종목".format(model_name))

In [31]:
MyKmeans.visualize_3d_with_answer(filtered_data["2020-01"],
                                  model2_save_dict["2020-01"]["model"],
                                 answer_data=label_data_pp_war["2020-01"],
                                 fig_title="{}_투자주의환기".format(model_name))

,PC1,PC2,PC3,label,answer
A000040,0.294115,0.063551,-0.374174,1,2.5
A000440,-0.565889,0.103335,-0.231612,2,2.5
A000650,-0.428699,-0.128830,0.169912,0,2.5
A000890,0.196051,0.104890,-0.350403,2,2.5
A000950,0.498525,-0.262781,0.119566,1,2.5
...,...,...,...,...,...
A317850,0.027396,-0.370472,-0.149895,0,2.5
A317870,-0.370508,-0.002331,-0.430695,2,2.5
A318000,-0.863809,0.274112,-0.082860,2,2.5
A322510,0.099560,-0.243867,0.175884,0,2.5
